In [ ]:
import logging as L
import os
import random
import subprocess
import sys
import duckdb
import numpy as np
import pandas as pd
from dataclasses import dataclass
from datetime import datetime
from itertools import product
from pathlib import Path, PosixPath
from typing import Optional, Tuple, List

import modal
import torch
import torch.nn.functional as F
import torch_frame as tf
from torch_frame.data import Dataset
from torch_frame.utils import infer_df_stype
from torch_frame.data.loader import DataLoader
from torch_frame.nn.encoder import EmbeddingEncoder, LinearEncoder
from torch_frame.nn.models import FTTransformer
from torchmetrics import Accuracy, MeanAbsoluteError

# Modal app
app        = modal.App("parcel-hp-sweep")
MINUTES     = 60
HOURS       = 60 * MINUTES
GPU_TYPE    = "T4"
vol         = modal.Volume.from_name("parcel", create_if_missing=True)
VPATH       = PosixPath("/vol")
MODEL_DIR   = VPATH / "models"

# container images
base_image  = modal.Image.debian_slim(python_version="3.11").uv_pip_install(
    "duckdb", "pyarrow", "pandas", "numpy"
)
torch_image = (
    base_image
    .uv_pip_install(
        "torch",
        "torch_frame",
        "torchmetrics",
        "wandb",
    )
)

# helpers
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

@dataclass
class HParams:
    # model
    channels: int        = 32
    n_layers: int        = 3
    # optimisation
    lr: float            = 1e-2
    # dataset
    sample_rows: int     = 10
    batch_size: int      = 5
    val_split: float     = 0.2
    seed: int            = 42

# data module
def build_dataloaders(
    parquet_path: str,
    target_col: str,
    task: str,
    h: HParams,
) -> Tuple[DataLoader, DataLoader, tf.data.Dataset.col_stats, dict]:
    """
    Returns train & val DataLoaders plus col_stats / col_names_dict needed by model.
    `task` ∈ {"regression", "binary"}
    """
    q = f"SELECT * FROM '{parquet_path}'"
    if h.sample_rows:
        q += f" USING SAMPLE reservoir({h.sample_rows} ROWS) REPEATABLE ({h.seed})"
    df = duckdb.sql(q).df()

    if task == "regression":
        df[target_col] = df[target_col].astype(float)
    else:
        df[target_col] = df[target_col].astype(int)

    # simple heuristic – drop obvious identifiers
    df = df.drop(columns=[c for c in df.columns if c.endswith("_id")], errors="ignore")

    col2stype = infer_df_stype(df)
    col2stype[target_col] = tf.numerical if task == "regression" else tf.categorical

    full_ds = Dataset(df, col_to_stype=col2stype, target_col=target_col)
    n_train = int(len(full_ds) * (1 - h.val_split))
    train_ds, val_ds = full_ds[:n_train], full_ds[n_train:]

    train_ds.materialize(path=f"{VPATH}/train_stats.pt")
    val_ds.materialize(path=f"{VPATH}/val_stats.pt", col_stats=train_ds.col_stats)

    train_loader = DataLoader(train_ds, batch_size=h.batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=h.batch_size)

    return train_loader, val_loader, train_ds.col_stats, train_ds.tensor_frame.col_names_dict

# training loop
def run_epoch(
    model, loader, optim, criterion, device
) -> float:
    model.train()
    total = n = 0.0
    for batch in loader:
        batch = batch.to(device)
        pred  = model(batch).squeeze()
        loss  = criterion(pred, batch.y.float())
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += loss.item() * len(batch.y)
        n += len(batch.y)
    return total / n

@torch.no_grad()
def evaluate(model, loader, metric, device) -> float:
    metric.reset()
    model.eval()
    for batch in loader:
        batch = batch.to(device)
        pred  = model(batch).squeeze()
        if isinstance(metric, Accuracy):
            pred = torch.sigmoid(pred)
        metric.update(pred, batch.y.float())
    return metric.compute().item()

# remote fn
@app.function(
    gpu=GPU_TYPE,
    image=torch_image,
    timeout=2 * HOURS,
    volumes={VPATH: vol},
    secrets=[modal.Secret.from_name("wandb-secret")]
)
def train_model(
    parquet_path: str,
    target_col: str,
    task: str,
    h: HParams,
    run_to_end: bool,
    max_epochs: int = 50,
):
    """
    Train one model with given hyper-parameters.
    """
    set_seed(h.seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # data
    train_loader, val_loader, col_stats, col_names_dict = build_dataloaders(
        parquet_path, target_col, task, h
    )

    # model
    enc = {
        tf.stype.categorical: EmbeddingEncoder(),
        tf.stype.numerical:   LinearEncoder(),
    }
    model = FTTransformer(
        channels=h.channels,
        num_layers=h.n_layers,
        out_channels=1,
        col_stats=col_stats,
        col_names_dict=col_names_dict,
        stype_encoder_dict=enc,
    )
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    model.to(device)

    # optimisation
    optim = torch.optim.AdamW(model.parameters(), lr=h.lr)
    criterion = torch.nn.SmoothL1Loss() if task == "regression" else torch.nn.BCEWithLogitsLoss()
    metric    = MeanAbsoluteError().to(device) if task == "regression" else Accuracy(task="binary").to(device)

    # W&B
    import wandb
    wandb.init(
        project="parcel-tabular",
        config=dict(**h.__dict__, task=task),
        name=f"E{datetime.now().strftime('%m%d_%H%M%S')}_ch{h.channels}_L{h.n_layers}_lr{h.lr:g}",
    )
    best_val, patience, PATIENCE = (np.inf if task=="regression" else 0.0), 0, 5

    for epoch in range(1, max_epochs + 1):
        tr_loss = run_epoch(model, train_loader, optim, criterion, device)
        val_metric = evaluate(model, val_loader, metric, device)
        wandb.log({"epoch": epoch, "train_loss": tr_loss,
                   "val_metric": val_metric})

        improved = (val_metric < best_val) if task=="regression" else (val_metric > best_val)
        if improved:
            best_val, patience = val_metric, 0
            # save weights into the shared Volume so other jobs can see them
            MODEL_DIR.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), MODEL_DIR / f"{wandb.run.name}.pt")
            vol.commit()
        else:
            patience += 1
            if patience >= PATIENCE and not run_to_end:
                break
    wandb.finish()
    return float(best_val)

# sweep main
@app.local_entrypoint()
def main(
    parquet_path: str = "../data/parcel_tracking_output_data.parquet",
    target_col: str   = "total_hours_from_receiving_to_last_success_delivery",
    task: str         = "regression",
):
    """
    Kick off an 8-run grid search, find the best, then resume it to completion.
    """
    # default hparams
    ref = HParams()
    ch_opts  = (16, ref.channels)
    lyr_opts = (2,  ref.n_layers)
    lr_opts  = (1e-3, ref.lr)

    hp_list: List[HParams] = [
        HParams(channels=channel, n_layers=layer, lr=lr,
                sample_rows=ref.sample_rows,
                batch_size=ref.batch_size,
                val_split=ref.val_split,
                seed=42)
        for channel, layer, lr in product(ch_opts, lyr_opts, lr_opts)
    ]

    print(f"Launching {len(hp_list)} hyper-param jobs on Modal …")
    # first round – early stop
    early_results = train_model.starmap(
        [
            (parquet_path, target_col, task, h, False)   # run_to_end=False
            for h in hp_list
        ],
        order_outputs=False,
    )

    best_h, best_val = None, np.inf if task=="regression" else -np.inf
    for res, h in zip(early_results, hp_list):
        v = float(res)
        better = (v < best_val) if task=="regression" else (v > best_val)
        print(f"{h} ⇒ {v:.4f}")
        if better:
            best_h, best_val = h, v

    print(f"\nBest so far: {best_h} ({best_val:.4f}) — continuing to full training …")
    # train_model.remote(parquet_path, target_col, task, best_h, True, max_epochs=100)

In [2]:
# tabular_experiments.py
# Re‑usable pipeline for ETA regression & POD classification
import argparse
import random
import duckdb
import numpy as np
import torch
# import torch.nn.functional as F
import torch_frame as tf
from torch_frame.data import Dataset
from torch_frame.utils import infer_df_stype
from torch_frame.data.loader import DataLoader
from torchmetrics import Accuracy, MeanAbsoluteError   # torchmetrics ≥1.4
from torch_frame.nn.encoder import EmbeddingEncoder, LinearEncoder
from torch_frame.nn.models import FTTransformer


In [3]:
# 0. 100 % reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 1. Data module (handles leakage‑free materialisation)
class TabularDataModule:
    def __init__(self, file_path: str, target: str, task: str,
                 sample_rows: int = None, batch_size: int = 256,
                 val_split: float = 0.2, seed: int = 42):
        """
        task ∈ {"regression", "binary"}
        """
        self.file_path, self.target, self.task = file_path, target, task
        self.sample_rows, self.batch_size, self.val_split, self.seed = \
            sample_rows, batch_size, val_split, seed

    def load(self):
        q = f"SELECT * FROM '{self.file_path}'"
        if self.sample_rows:
            q += f" USING SAMPLE reservoir({self.sample_rows} ROWS) REPEATABLE ({self.seed})"
        df = duckdb.sql(q).df()

        # Cast target dtype & drop high‑cardinality IDs
        if self.task == "regression":
            df[self.target] = df[self.target].astype(float)
        else:
            df[self.target] = df[self.target].astype(int)
        df = df.drop(columns=["customer_id", "parcel_id"], errors="ignore")

        # Detect column semantics *before* split
        col_to_stype = infer_df_stype(df)
        col_to_stype[self.target] = tf.numerical if self.task == "regression" else tf.categorical

        # Create Dataset & materialise **train only** to avoid leakage
        ds = Dataset(df, col_to_stype=col_to_stype, target_col=self.target)
        n_train = int(len(ds) * (1 - self.val_split))
        train_ds, val_ds = ds[:n_train], ds[n_train:]
        train_ds.materialize(path="cache/train.pt")             # persisted stats
        val_ds.materialize(path="cache/val.pt", col_stats=train_ds.col_stats)

        # Mini‑batch loaders
        self.train_loader = DataLoader(train_ds, batch_size=self.batch_size, shuffle=True)
        self.val_loader   = DataLoader(val_ds,   batch_size=self.batch_size)

        self.col_stats, self.col_names_dict = train_ds.col_stats, train_ds.tensor_frame.col_names_dict


# 2. Model factory (drop your own backbones here)
def build_model(name: str, out_channels: int,
                col_stats, col_names_dict, stype_enc):
    if name == "ftt":
        return FTTransformer(channels=32, out_channels=out_channels,
                             num_layers=4,
                             col_stats=col_stats,
                             col_names_dict=col_names_dict,
                             stype_encoder_dict=stype_enc)
    elif name == "mlp":        # tiny baseline for sanity checks
        from torch_frame.nn.models import MLP
        return MLP(channels=64, out_channels=out_channels,
                   col_stats=col_stats,
                   col_names_dict=col_names_dict,
                   stype_encoder_dict=stype_enc)
    else:
        raise ValueError(f"Unknown model {name}")


# 3. Generic trainer
def train(model, loader, optimizer, criterion):
    model.train()
    loss_sum = 0
    n = 0
    for batch in loader:
        pred = model(batch).squeeze()
        y = batch.y.float()
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item() * len(y)
        n += len(y)
    return loss_sum / n

@torch.no_grad()
def evaluate(model, loader, metric):
    model.eval()
    metric.reset()
    for batch in loader:
        pred = model(batch).squeeze()
        y = batch.y.float()
        if isinstance(metric, Accuracy):
            pred = torch.sigmoid(pred)
        metric.update(pred, y)
    return metric.compute().item()


# 4. Experiment runner
def run(args):
    set_seed(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Build data
    dm = TabularDataModule(args.file, args.target, args.task,
                           sample_rows=args.sample, batch_size=args.batch,
                           val_split=args.val_split, seed=args.seed)
    dm.load()

    # Shared stype encoders
    stype_enc = {tf.stype.categorical: EmbeddingEncoder(),
                 tf.stype.numerical:   LinearEncoder()}




    # Model & optimiser
    out_dim = 1    # both tasks output scalar
    model = build_model(args.model, out_dim,
                        dm.col_stats, dm.col_names_dict, stype_enc).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr)

    # Loss & metric
    if args.task == "regression":
        criterion = torch.nn.SmoothL1Loss()
        metric = MeanAbsoluteError().to(device)
    else:
        criterion = torch.nn.BCEWithLogitsLoss()
        metric = Accuracy(task="binary").to(device)

    # Training loop with rudimentary early stopping
    best_val, patience = float("inf") if args.task == "regression" else 0.0, 0
    for epoch in range(1, args.epochs + 1):
        tr_loss = train(model, dm.train_loader, opt, criterion)
        val_metric = evaluate(model, dm.val_loader, metric)
        print(f"E{epoch:02d} | train-loss {tr_loss:.4f} | val‑{'MAE' if args.task == 'regression' else 'Acc'} {val_metric:.4f}")
        improve = (val_metric < best_val) if args.task == "regression" else (val_metric > best_val)
        if improve:
            best_val, patience = val_metric, 0
            torch.save(model.state_dict(), "best.pt")
        else:
            patience += 1
            if patience >= args.patience:
                break

In [4]:
run()

TypeError: run() missing 1 required positional argument: 'args'

In [ ]:
# 5. CLI – define once; compare infinitely
if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("--file",   type=str, required=True)
    p.add_argument("--target", type=str, required=True)
    p.add_argument("--task",   choices=["regression", "binary"], required=True)
    p.add_argument("--model",  choices=["ftt", "mlp"], default="ftt")
    p.add_argument("--sample", type=int, default=None)
    p.add_argument("--batch",  type=int, default=256)
    p.add_argument("--lr",     type=float, default=5e-3)
    p.add_argument("--epochs", type=int, default=50)
    p.add_argument("--val_split", type=float, default=0.2)
    p.add_argument("--patience",  type=int, default=5)
    p.add_argument("--seed",      type=int, default=42)
    # run(p.parse_args())

In [13]:
import duckdb
import torch
import torch.nn.functional as F
import torch_frame as tf
from torch_frame.data import Dataset
from torch_frame.utils import infer_df_stype
from torch_frame.data.loader import DataLoader
from torch_frame.nn.encoder import EmbeddingEncoder, LinearEncoder
from torch_frame.nn.models import FTTransformer   # any backbone works

# 1. Read & sample exactly the same way you did
file_path = "../data/parcel_tracking_output_data.parquet"
df = duckdb.sql(f"""
    SELECT * FROM '{file_path}'
    USING SAMPLE reservoir(100 ROWS)
    REPEATABLE (100)
""").df()

In [ ]:
# import duckdb
# import torch
# import torch.nn.functional as F
# import torch_frame as tf
# from torch_frame.data import Dataset
# from torch_frame.utils import infer_df_stype
# from torch_frame.data.loader import DataLoader
# from torch_frame.nn.encoder import EmbeddingEncoder, LinearEncoder
# from torch_frame.nn.models import FTTransformer   # any backbone works

# # 1. Read & sample exactly the same way you did
# file_path = "../data/parcel_tracking_output_data.parquet"
# df = duckdb.sql(f"""
#     SELECT * FROM '{file_path}'
#     USING SAMPLE reservoir(100 ROWS)
#     REPEATABLE (100)
# """).df()

# # Targets ----------------------------------------------------
# ETA_COL   = "total_hours_from_receiving_to_last_success_delivery"
# POD_COL   = "is_successful_delivery"

# # Cast targets to numeric
# df[ETA_COL] = df[ETA_COL].astype(float)
# df[POD_COL] = df[POD_COL].astype(int)
# df.drop(columns=["customer_id", "parcel_id"], inplace=True)

# eta_df = df.drop(columns=[POD_COL])
# pod_df = df.drop(columns=[ETA_COL])

# # 2. Build two Dataset objects (one per task)
# eta_col_to_stype = infer_df_stype(eta_df)
# eta_col_to_stype[ETA_COL] = tf.numerical

# pod_col_to_stype = infer_df_stype(pod_df)
# pod_col_to_stype[POD_COL] = tf.categorical

# eta_ds = Dataset(eta_df, col_to_stype=eta_col_to_stype, target_col=ETA_COL)
# pod_ds = Dataset(pod_df, col_to_stype=pod_col_to_stype, target_col=POD_COL)

# # Materialise once so we can reuse stats & mappings
# eta_ds.materialize()              # gives .tensor_frame, .col_stats
# pod_ds.materialize()

# # Split
# train_eta = eta_ds[:0.8]    # same slice trick as in quick‑tour :contentReference[oaicite:3]{index=3}
# val_eta   = eta_ds[0.8:]
# train_pod = pod_ds[:0.8]
# val_pod   = pod_ds[0.8:]

# # 3.  Mini‑batch loaders
# BATCH = 5
# train_eta_loader = DataLoader(train_eta, batch_size=BATCH, shuffle=True)
# val_eta_loader   = DataLoader(val_eta,   batch_size=BATCH)

# train_pod_loader = DataLoader(train_pod, batch_size=BATCH, shuffle=True)
# val_pod_loader   = DataLoader(val_pod,   batch_size=BATCH)

# # 4. Common stype‑wise encoders
# stype_enc = {
#     tf.stype.categorical: EmbeddingEncoder(),
#     tf.stype.numerical:   LinearEncoder(),
# }

# # 5‑A. ETA regressor  (FT‑Transformer, 1 output)
# eta_model = FTTransformer(
#     channels=32,
#     out_channels=1,
#     num_layers=3,
#     col_stats=train_eta.col_stats,
#     col_names_dict=train_eta.tensor_frame.col_names_dict,
#     stype_encoder_dict=stype_enc,
# )
# opt_eta = torch.optim.AdamW(eta_model.parameters(), lr=5e-3)

# # 5‑B. POD classifier (same backbone, sigmoid output)
# pod_model = FTTransformer(
#     channels=32,
#     out_channels=1,
#     num_layers=3,
#     col_stats=train_pod.col_stats,
#     col_names_dict=train_pod.tensor_frame.col_names_dict,
#     stype_encoder_dict=stype_enc,
# )
# opt_pod = torch.optim.AdamW(pod_model.parameters(), lr=1e-3)

# # 6. Plain PyTorch training loops (no Lightning/Rich headaches)
# def train_epoch(model, loader, optimizer, is_regression: bool):
#     model.train()
#     total = 0
#     loss_sum = 0
#     loss_history = []
#     for tf_batch in loader:
#         pred = model(tf_batch)
#         target = tf_batch.y.float()
#         loss = F.smooth_l1_loss(pred.squeeze(), target) if is_regression else F.binary_cross_entropy_with_logits(pred.squeeze(), target)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         loss_history.append(loss.item())
#         loss_sum += loss.item() * len(target)
#         total += len(target)
#     return loss_sum / total, loss_history

# for epoch in range(1, 30):
#     eta_loss, eta_loss_history = train_epoch(eta_model, train_eta_loader, opt_eta, True)
#     pod_loss, pod_loss_history = train_epoch(pod_model, train_pod_loader, opt_pod, False)
#     print(f"Epoch {epoch:02d} | ETA MAE-ish {eta_loss:.3f} | POD BCE {pod_loss:.3f}")

# # 7.  Validation & inference# -----------------------------------------------------------
# @torch.no_grad()
# def evaluate(model, loader, is_regression):
#     model.eval()
#     outs, ys = [], []
#     for tf_batch in loader:
#         outs.append(model(tf_batch).cpu())
#         ys.append(tf_batch.y.float().cpu())
#     pred = torch.cat(outs).squeeze()
#     y    = torch.cat(ys).squeeze()
#     if is_regression:
#         return torch.mean(torch.abs(pred - y)).item()      # MAE
#     else:
#         prob = torch.sigmoid(pred)
#         acc  = ((prob > 0.5) == y.bool()).float().mean()
#         return acc.item()

# print("ETA MAE (val):", evaluate(eta_model, val_eta_loader, True))
# print("POD ACC (val):", evaluate(pod_model, val_pod_loader, False))

In [52]:
# torch.manual_seed(42)
# pod_model.train()
# loss_history = []
# for epoch in range(0, 50):
#     print("")
#     print(f"EPOCH {epoch}:")
#     for idx, batch in enumerate(train_pod_loader):
#         print(batch)
#         pred = pod_model(batch)
#         target = batch.y.float()
#         loss = F.smooth_l1_loss(pred.squeeze(), target) if False else F.binary_cross_entropy_with_logits(pred.squeeze(), target)
#         opt_eta.zero_grad()
#         loss.backward()
#         opt_eta.step()
#         loss_history.append(loss.item())
#         print(f"step {idx}: {loss.item()}")

In [ ]:
# for epoch in range(1, 30):
#     eta_loss, eta_loss_history = train_epoch(eta_model, train_eta_loader, opt_eta, True)
#     pod_loss, pod_loss_history = train_epoch(pod_model, train_pod_loader, opt_pod, False)
#     print(f"Epoch {epoch:02d} | ETA MAE-ish {eta_loss:.3f} | POD BCE {pod_loss:.3f}")

Epoch 01 | ETA MAE-ish 24.736 | POD BCE 0.581
Epoch 02 | ETA MAE-ish 23.274 | POD BCE 0.405
Epoch 03 | ETA MAE-ish 21.756 | POD BCE 0.373
Epoch 04 | ETA MAE-ish 19.835 | POD BCE 0.331
Epoch 05 | ETA MAE-ish 17.341 | POD BCE 0.270
Epoch 06 | ETA MAE-ish 14.937 | POD BCE 0.244
Epoch 07 | ETA MAE-ish 13.276 | POD BCE 0.174
Epoch 08 | ETA MAE-ish 13.048 | POD BCE 0.224
Epoch 09 | ETA MAE-ish 12.977 | POD BCE 0.166
Epoch 10 | ETA MAE-ish 12.933 | POD BCE 0.211
Epoch 11 | ETA MAE-ish 13.068 | POD BCE 0.137
Epoch 12 | ETA MAE-ish 12.984 | POD BCE 0.110
Epoch 13 | ETA MAE-ish 13.015 | POD BCE 0.104
Epoch 14 | ETA MAE-ish 12.997 | POD BCE 0.105
Epoch 15 | ETA MAE-ish 12.969 | POD BCE 0.080
Epoch 16 | ETA MAE-ish 12.974 | POD BCE 0.073
Epoch 17 | ETA MAE-ish 12.979 | POD BCE 0.066
Epoch 18 | ETA MAE-ish 12.965 | POD BCE 0.067
Epoch 19 | ETA MAE-ish 13.057 | POD BCE 0.063
Epoch 20 | ETA MAE-ish 12.859 | POD BCE 0.060
Epoch 21 | ETA MAE-ish 12.922 | POD BCE 0.060
Epoch 22 | ETA MAE-ish 12.936 | PO

In [ ]:
# # 7.  Validation & inference# -----------------------------------------------------------
# @torch.no_grad()
# def evaluate(model, loader, is_regression):
#     model.eval()
#     outs, ys = [], []
#     for tf_batch in loader:
#         outs.append(model(tf_batch).cpu())
#         ys.append(tf_batch.y.float().cpu())
#     pred = torch.cat(outs).squeeze()
#     y    = torch.cat(ys).squeeze()
#     if is_regression:
#         return torch.mean(torch.abs(pred - y)).item()      # MAE
#     else:
#         prob = torch.sigmoid(pred)
#         acc  = ((prob > 0.5) == y.bool()).float().mean()
#         return acc.item()

# print("ETA MAE (val):", evaluate(eta_model, val_eta_loader, True))
# print("POD ACC (val):", evaluate(pod_model, val_pod_loader, False))

ETA MAE (val): 24.087116241455078
POD ACC (val): 0.949999988079071
